<a href="https://colab.research.google.com/github/deanhoperobertson/NLP-Text-Classifiation-/blob/master/Text_Classification_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Amazon Reviews 

In [0]:
import urllib.request
import pandas as pd
import urllib
from sklearn import model_selection, preprocessing

#Feature Engineering
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

#keras
from keras.preprocessing.text import Tokenizer


In [0]:
#import data from my github repo
url = "https://raw.githubusercontent.com/deanhoperobertson/NLP-Text-Classifiation-/master/corpus.txt"
data = urllib.request.urlopen(url).read()
data = data.decode('utf-8')


In [0]:
labels, texts = [], []
for i, line in enumerate(str(data).split("\n")):
    content = line.replace("\\","").split()
    labels.append(content[0])
    texts.append(" ".join(content[1:]))

# create a dataframe using texts and lables
trainDF = pd.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels

In [13]:
trainDF.head()

,text,label
0,Stuning even for the non-gamer: This sound tra...,__label__2
1,The best soundtrack ever to anything.: I'm rea...,__label__2
2,Amazing!: This soundtrack is my favorite music...,__label__2
3,Excellent Soundtrack: I truly like this soundt...,__label__2
4,"Remember, Pull Your Jaw Off The Floor After He...",__label__2


In [0]:
#split into train and test set (default is 25%)
#set seed for consistency
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'], test_size=0.25, random_state=1)


In [0]:
# encode target labels into intergers between 0 - no.classes
# label 2 = 1
# label 1 = 0
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

## Feature Engineeering

Now we need to convert the worded sentences into vectors by replacing the words with numercil representation. 

for Example:
"The cat sat on the hat" = {1,3,10,7,1,12}

This can be done in a number of ways such as:
-  CountVectorizer (Bag of Words)
- TF-IDF Matrix



## 1.1 Bag of Words

Here we use countervectorize to create a "bag of words", that is a corpus of unique words from the text that we are interested in modeling. The CountVectorizer's defaults settings:
- Convert all characters into lower case
- Remove all stops words such as “the”, “a”, “an”, “in” 

This bag of words is then used to vectorize each sentence (row) is a document term matrix with the columns defined a specific term/word.

- fit() function - learns a vocabulary from text
- transform() function - encodes the documents into vectors 


**Pro's**
- Easy to undertand and apply
- Flexible vocabulary

**Cons**
- All words are treated eqaully (include meanlingless words)
-  Discarding word order ignores the context, which can be very valuable in text model. 


![alt text](Bag of Words.png "Title")

### 1.1.1 CountVectorizer

In [0]:
# instantiate a count vectorizer object to tokenize on a word level
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', strip_accents ="ascii", stop_words=None)
count_vect.fit(trainDF['text'])

#see the unique words being tokenized 
count_vect.get_feature_names()[1:10]

#create a document term matrix using the 
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [17]:
#Here we can see the how the each word have been tokenized
print(count_vect.vocabulary_)

{'stuning': 27060, 'even': 10124, 'for': 11399, 'the': 28063, 'non': 19152, 'gamer': 11934, 'this': 28205, 'sound': 26182, 'track': 28706, 'was': 30586, 'beautiful': 3084, 'it': 15158, 'paints': 20158, 'senery': 24895, 'in': 14325, 'your': 31500, 'mind': 17998, 'so': 25958, 'well': 30761, 'i': 14019, 'would': 31269, 'recomend': 22885, 'to': 28474, 'people': 20569, 'who': 30897, 'hate': 13145, 'vid': 30208, 'game': 11924, 'music': 18640, 'have': 13171, 'played': 21109, 'chrono': 5408, 'cross': 7048, 'but': 4400, 'out': 19887, 'of': 19506, 'all': 1487, 'games': 11936, 'ever': 10135, 'has': 13123, 'best': 3319, 'backs': 2768, 'away': 2668, 'from': 11712, 'crude': 7072, 'keyboarding': 15705, 'and': 1741, 'takes': 27677, 'a': 710, 'fresher': 11659, 'step': 26747, 'with': 31081, 'grate': 12611, 'guitars': 12823, 'soulful': 26174, 'orchestras': 19746, 'impress': 14292, 'anyone': 1942, 'cares': 4727, 'listen': 16630, '_': 698, 'soundtrack': 26198, 'anything': 1945, 'm': 17014, 'reading': 22723

In [18]:
#view that DT matrix in dataframe
pd.DataFrame(xtrain_count.toarray(), columns=count_vect.get_feature_names()).head()

,0,00,000,001,002,00290,007,0070412901,0072316373,008,...,zzzz,zzzzz,zzzzzz,zzzzzzz,zzzzzzzzzz,zzzzzzzzzzzz,zzzzzzzzzzzzz,zzzzzzzzzzzzzzzzzz,zzzzzzzzzzzzzzzzzzzzz,zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 1.1.2 Bag of Words ( with Keras)

In [0]:
#tokenizes the words - here we aree able to define the maxium number of words we would like to include ranked by frequency
t = Tokenizer(num_words=None)
# fit the tokenizer on the documents
t.fit_on_texts(trainDF['text'])

In [72]:
#see vectorization of words
print(t.word_counts)

OrderedDict([('stuning', 1), ('even', 1429), ('for', 8376), ('the', 41321), ('non', 142), ('gamer', 4), ('this', 15515), ('sound', 484), ('track', 193), ('was', 7395), ('beautiful', 272), ('it', 16503), ('paints', 7), ('senery', 1), ('in', 9976), ('your', 1742), ('mind', 247), ('so', 3356), ('well', 1500), ('i', 20914), ('would', 2396), ('recomend', 40), ('to', 19341), ('people', 906), ('who', 1598), ('hate', 115), ('vid', 2), ('game', 731), ('music', 880), ('have', 4925), ('played', 181), ('chrono', 5), ('cross', 41), ('but', 5982), ('out', 2230), ('of', 16897), ('all', 3284), ('games', 140), ('ever', 949), ('has', 2122), ('best', 1129), ('backs', 3), ('away', 384), ('from', 2517), ('crude', 22), ('keyboarding', 1), ('and', 21559), ('takes', 235), ('a', 20632), ('fresher', 2), ('step', 72), ('with', 5657), ('grate', 1), ('guitars', 17), ('soulful', 7), ('orchestras', 1), ('impress', 17), ('anyone', 545), ('cares', 27), ('listen', 255), ('soundtrack', 90), ('anything', 343), ("i'm", 96

In [71]:
# Easy method to calculate vocabulary (unique word count)
len(set(t.word_counts))


33470

In [80]:
xtrain_keras = t.texts_to_matrix(train_x, mode='count')
xvalid_keras = t.texts_to_matrix(valid_x, mode='count')
print(xtrain_keras)

[[ 0. 11.  1. ...  0.  0.  0.]
 [ 0.  1.  1. ...  0.  0.  0.]
 [ 0.  3.  0. ...  0.  0.  0.]
 ...
 [ 0.  3.  1. ...  0.  0.  0.]
 [ 0.  2.  1. ...  0.  0.  0.]
 [ 0.  6.  3. ...  0.  0.  0.]]


##1.2 TD-IDF Vectors 




A TDIF matrix is an alternative method of representing  words in tokenized form. This method differs from the traditional bag of words model as it weights the importance of each word different depending on its frequency within the list of documents. 

This allows for words like “the” which generally appear many times (large counts) and are not very meaningful in the encoded vectors. The TD-IFD vectorization method weights these meaningless words less than other words that offer more value. TF-IDF score is composed by two terms:

- Normalized Term Frequency (TF)
- Inverse Document Frequency (IDF)

with the following formula:

\begin{align}
\frac{Term Frequency​(TF)}{Inverse Document Frequency​(IDF)}
\end{align}

TF component summarizes how often a given word appears within a document, and the IDF component reduces the weighting of words that appear a lot across documents. This is done using by computing  logarithm (Ln) of the number of the documents in the corpus divided by the number of documents where the specific term appears.

The end result is method of vectorizing text that **highlights** words that are more interesting within a collections of texts. 

**Pros**
- Accounts for different word importance
- small range of values in matrix speeds up convergance

**Cons**
- Discards word order ignores and thus looses context.



### 1.2.1 Word-Level

In [0]:
# vectorizing on a word level
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

In [28]:
# The first line shows the vectoriztion of vocab
# The idf score indicates how frequent the word is. The lowest score of 1.0 to the most frequently observed word
print(tfidf_vect.vocabulary_)
print(tfidf_vect.idf_)

{'even': 1540, 'for': 1806, 'the': 4420, 'non': 3001, 'this': 4449, 'sound': 4106, 'track': 4543, 'was': 4811, 'beautiful': 438, 'it': 2384, 'in': 2262, 'your': 4992, 'mind': 2848, 'so': 4064, 'well': 4851, 'i': 2218, 'would': 4949, 'recomend': 3576, 'to': 4496, 'people': 3210, 'who': 4874, 'hate': 2061, 'game': 1880, 'music': 2928, 'have': 2067, 'played': 3281, 'cross': 1064, 'but': 638, 'out': 3112, 'of': 3046, 'all': 209, 'games': 1882, 'ever': 1545, 'has': 2057, 'best': 480, 'away': 375, 'from': 1852, 'crude': 1067, 'and': 247, 'takes': 4334, 'a': 88, 'step': 4186, 'with': 4911, 'guitars': 2014, 'impress': 2255, 'anyone': 271, 'cares': 691, 'listen': 2630, 'soundtrack': 4110, 'anything': 272, 'm': 2702, 'reading': 3547, 'lot': 2681, 'reviews': 3700, 'saying': 3813, 'that': 4418, 'is': 2377, 'figured': 1716, 'd': 1092, 'write': 4957, 'review': 3695, 'disagree': 1262, 'bit': 503, 'my': 2934, 's': 3779, 'ultimate': 4626, 'masterpiece': 2768, 'timeless': 4484, 'been': 450, 'listening':

### 1.2.2 N-gram Level (Basic)

A further level of vectorization is adapting an N-gram approach which processes text as a sequence of words. Simply put, an N-gram is a sequence of N words. N-grams can be any size greater than one word. For Example:

"Doctor" - Uni-gram <br>
"Doctor has" = Bi-gram" <br>
"Doctor has evidence" - Tri-gram <br>

The N-gram approach cater for the fact that some words are often found in close proximitry to each other and together can have a very different meaning processed individually. For example, the bi-gram "not good" has a very different meaning if processed as "not" & "good" on a word level. The N-gram model allows for these words relationships to be reckonized within the text. 

In most cases, we set an N-gram range that allow a variety of sized N-grams a chance to be presented. 


In [0]:
# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

##1.3 Customize a Vectorization Function

In [0]:
#only take the top 2000 N-grams and 
#ignore words that a doc freq of less than 2 
TOP_K = 20000
MIN_DOC_FREQ = 2

def ngram_vectorize(train_texts, train_labels, val_texts):
    kwargs = {
        'ngram_range' : (1, 2),
        'dtype' : 'int32',
        'strip_accents' : 'unicode',
        'decode_error' : 'replace',
        'analyzer' : 'word',
        'min_df' : MIN_DOC_FREQ,
    }
    
    # Learn Vocab from train texts and vectorize train and val sets
    tfidf_vectorizer = TfidfVectorizer(**kwargs)
    x_train = tfidf_vectorizer.fit_transform(train_texts)
    x_val = tfidf_vectorizer.transform(val_texts)
    
    # Select best k features, with feature importance measured by f_classif (which is the deafault)
    selector = SelectKBest(f_classif, k=min(TOP_K, x_train.shape[1]))  
    selector.fit(x_train, train_labels)
    x_train = selector.transform(x_train).astype('float32')
    x_val = selector.transform(x_val).astype('float32')
    return x_train, x_val

In [89]:
    # Vectorize the data and returns a tf-idf matrix
    x_train, x_val = ngram_vectorize(train_x, train_y, valid_x)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:1569: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)
